In [14]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

### Take new image and resize to (300, 400, 3)

In [16]:
prefix = './backgrounds'
background_names = np.sort(os.listdir(prefix)).tolist()

In [17]:
background_names = [name for name in background_names if ('.jpg' or '.png') in name]
background_names

['black.jpg',
 'blue.jpg',
 'brown.jpg',
 'green.jpg',
 'multi.jpg',
 'orange.jpg',
 'purple.jpg',
 'white.jpg',
 'yellow.jpg']

In [18]:
for name in background_names:
    img = cv2.imread(prefix + '/' + name)
    print(prefix + '/' + name)
    chs = img.shape[2]
    like_shape = (300, 400, chs)
    img_resized = np.zeros(like_shape)
    for i in range(chs):
        img_resized[:,:,i] = cv2.resize(img[:,:,i], (400,300))
    new_name = Path(name).stem
    cv2.imwrite(f'./backgrounds_300x400/{new_name}.jpg', img_resized)

./backgrounds/black.jpg
./backgrounds/blue.jpg
./backgrounds/brown.jpg
./backgrounds/green.jpg
./backgrounds/multi.jpg
./backgrounds/orange.jpg
./backgrounds/purple.jpg
./backgrounds/white.jpg
./backgrounds/yellow.jpg


### Generate figures for colors

In [15]:
NUMBER_PER_COLOR = 15

In [16]:
circles_dir = './figures/images'
os.makedirs(circles_dir, exist_ok=True)
triangles_dir = './figures/masks'
os.makedirs(triangles_dir, exist_ok=True)

circles_dir = './dataset/images'
os.makedirs(circles_dir, exist_ok=True)
circles_dir = './dataset/masks'
os.makedirs(circles_dir, exist_ok=True)

In [17]:
# -green: (0,255,0)
# -black: (1, 1, 1)
# -blue: (0,0,255)
# -brown: (139,69,19)
# -multi: (255,20,147)
# -orange: (255,165,0)
# -purple: (138,43,226)
# -white: (254,254,254)
# -yellow: (255,255,0)

In [18]:
dict_colors = {'green': (1,128,1), 'black': (1, 1, 1), 'blue': (1,1,255), 
               'brown': (139,69,19), 'yellow': (255,255,1), 'multi': (255,20,147), 
               'orange': (255,165,1), 'purple': (138,43,226), 'white': (254,254,254)}

In [19]:
def circle_with_color(N, width, height, color):
    for j in range(N):
        img_circle = np.ones((width, height, 3), np.uint8) * 0
        mask_circle = np.ones((width, height, 3), np.uint8) * 0

        center_x = np.random.randint(100, 200, size=1)[0]
        center_y = np.random.randint(100, 300, size=1)[0]
        max_radius = min(min(center_x, center_y), min(width - center_x, height - center_y))
        radius = np.random.randint(50, max_radius)
        
        img_color = dict_colors[color]
        cv2.circle(img_circle,(center_x,center_y), radius, img_color, -1)  
        cv2.circle(mask_circle,(center_x,center_y), radius, (255,255,255), -1)

#         cv2.imwrite(f'./figures//images/{color}_circle_{j}.jpg', img_circle)
        np.save(f'./figures//images/{color}_circle_{j}.npy', img_circle)
        cv2.imwrite(f'./dataset/masks/{color}_circle_{j}.png', mask_circle)

In [20]:
for color in dict_colors.keys():
    circle_with_color(N=NUMBER_PER_COLOR, width=300, height=400, color=color)

In [21]:
def rectangle_with_color(N, width, height, color):
    for j in range(N):
        img_rectangle = np.ones((width, height, 3), np.uint8) * 0
        mask_rectangle = np.ones((width, height, 3), np.uint8) * 0

        low_left_point = np.random.randint(0, 250, size =2)
        max_size = max(min(low_left_point[0], low_left_point[1]), 
                       min(width - low_left_point[0], height - low_left_point[1]))

        width_rectangle = np.random.randint(50, max(150,max_size), size=1)
        height_rectangle = np.random.randint(50, max(150,max_size), size=1)

        upper_right_point = np.array([low_left_point[0] + width_rectangle, low_left_point[1] + height_rectangle])
        upper_right_point = upper_right_point.flatten()

        img_color = dict_colors[color]

        cv2.rectangle(img_rectangle,(low_left_point[0],low_left_point[1]),
                      (upper_right_point[0],upper_right_point[1]),img_color,-1)
        cv2.rectangle(mask_rectangle,(low_left_point[0],low_left_point[1]),
                      (upper_right_point[0],upper_right_point[1]),(255,255,255),-1)

#         cv2.imwrite(f'./figures/images/{color}_rectangle_{j}.jpg', img_rectangle)
        np.save(f'./figures/images/{color}_rectangle_{j}.npy', img_rectangle)
        cv2.imwrite(f'./dataset/masks/{color}_rectangle_{j}.png', mask_rectangle)

In [22]:
for color in dict_colors.keys():
    rectangle_with_color(N=NUMBER_PER_COLOR, width=300, height=400, color=color)

In [23]:
def triangle_with_color(N, width, height, color):
    for j in range(N):
        img_triangle = np.ones((width, height, 3), np.uint8) * 0
        mask_triangle = np.ones((width, height, 3), np.uint8) * 0

        pt1 = np.random.randint(50, 350, size=2)
        pt1 = (pt1[0], pt1[1])

        low_line = np.random.randint(50, 400 - pt1[0], size=1)
        pt2 = ((pt1[0] + low_line)[0], pt1[1])

        if pt1[1] < 400 - pt1[1]:

            pt3_y = pt1[1] + np.random.randint(pt1[1], 400 - pt1[1], size=1)
        else:
            pt3_y = pt1[1] - np.random.randint(50, pt1[1], size=1)
        pt3_x = np.random.randint(min(pt1[0], pt1[0] - 50), max(pt2[0], pt2[0]+50), size=1)

        pt3 = (pt3_x[0], pt3_y[0])

        triangle = np.array( [pt1, pt2, pt3] )

        img_color = dict_colors[color]

        cv2.drawContours(img_triangle, [triangle], 0, img_color, -1)
        cv2.drawContours(mask_triangle, [triangle], 0, (255,255,255), -1)

#         cv2.imwrite(f'./figures/images/{color}_triangle_{j}.jpg', img_triangle)
        np.save(f'./figures/images/{color}_triangle_{j}.npy', img_triangle)
        cv2.imwrite(f'./dataset/masks/{color}_triangle_{j}.png', mask_triangle)

In [24]:
for color in dict_colors.keys():
    triangle_with_color(N=NUMBER_PER_COLOR, width=300, height=400, color=color)

### Connect images by colors

In [25]:
def connect_imgs(path_background, path_front_img):
    background = cv2.imread(path_background)
#     front_img = cv2.imread(path_front_img)
    front_img = np.load(path_front_img)
    front_img = cv2.cvtColor(front_img, cv2.COLOR_BGR2RGB)
    name_img = path_front_img.split('/')[3].split('.')[0]
    print(name_img)
    img_common = np.zeros_like(background)
    for i in range(front_img.shape[0]):
        for j in range(front_img.shape[1]):
            for ch in range(front_img.shape[2]):
                if front_img[i][j][ch] != 0:
                    img_common[i][j][ch] = front_img[i][j][ch]
                else:
                    img_common[i][j][ch] = background[i][j][ch]
    cv2.imwrite(f'./dataset/images/{name_img}.jpg', img_common)

In [26]:
for color in dict_colors.keys():
    figure_list = ['circle', 'rectangle', 'triangle']
    path = './figures/images/' 
    pathes_color = []
    for figure in figure_list:
        for j in range(NUMBER_PER_COLOR):
            pathes_color.append(path + f'{color}_{figure}_{j}.npy')
            
    for path in pathes_color:
        connect_imgs(f'./backgrounds_300x400/{color}.jpg', path)

green_circle_0
green_circle_1
green_circle_2
green_circle_3
green_circle_4
green_circle_5
green_circle_6
green_circle_7
green_circle_8
green_circle_9
green_circle_10
green_circle_11
green_circle_12
green_circle_13
green_circle_14
green_rectangle_0
green_rectangle_1
green_rectangle_2
green_rectangle_3
green_rectangle_4
green_rectangle_5
green_rectangle_6
green_rectangle_7
green_rectangle_8
green_rectangle_9
green_rectangle_10
green_rectangle_11
green_rectangle_12
green_rectangle_13
green_rectangle_14
green_triangle_0
green_triangle_1
green_triangle_2
green_triangle_3
green_triangle_4
green_triangle_5
green_triangle_6
green_triangle_7
green_triangle_8
green_triangle_9
green_triangle_10
green_triangle_11
green_triangle_12
green_triangle_13
green_triangle_14
black_circle_0
black_circle_1
black_circle_2
black_circle_3
black_circle_4
black_circle_5
black_circle_6
black_circle_7
black_circle_8
black_circle_9
black_circle_10
black_circle_11
black_circle_12
black_circle_13
black_circle_14
black